In [23]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPEN_AI_API']=os.getenv('OPEN_API_KEY')


In [8]:
text="i am charani, a passionate AI Developer"

In [ ]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings(model="text-embedding-3-large",dimensions=300)
vector_result=embeddings.embed_query(text)
vector_result

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [11]:
from langchain_community.document_loaders import TextLoader

loader=TextLoader('speech.txt')
docs=loader.load()
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness 

In [12]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
final_documents=text_splitter.split_documents(docs)
final_documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': 'speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦'),
 Document(metadata={'source': 'speech.txt'}, page_

In [ ]:
from langchain_community.vectorstores import Chroma
db=Chroma.from_documents(final_documents,embeddings)
db

In [14]:
query="It will be all the easier for us to conduct ourselves as belligerents"
retrieved_results=db.similarity_search(query)
retrieved_results

NameError: name 'db' is not defined

Nice, you’ve reached the **core of RAG systems** now 🔥
Let’s unpack `Chroma.from_documents()` **slowly, structurally, and practically**—not just what it does, but *why* each parameter exists and how syntax changes in different cases.

---

## Your code

```python
from langchain_community.vectorstores import Chroma

db = Chroma.from_documents(final_documents, embeddings_1024)
db
```

---

# 1️⃣ What is Chroma?

**Chroma** is a **vector store database**.

### Meaning:

* Stores **embeddings (vectors)**
* Enables **semantic search**
* Used in **RAG (Retrieval Augmented Generation)**

Think of it as:

```
Text → Embedding (numbers) → Stored → Similarity Search
```

---

# 2️⃣ What does `from_documents()` do?

```python
Chroma.from_documents(documents, embedding, **kwargs)
```

### Internally, it does ALL of this:

1. Takes `Document` objects
2. Extracts `page_content`
3. Generates embeddings using `embeddings_1024`
4. Stores:

   * vectors
   * text
   * metadata
5. Creates a searchable vector index

👉 **One-line summary**

> Converts documents into vectors and stores them in ChromaDB.

---

# 3️⃣ Parameters explained (important)

### ① `documents`

```python
final_documents
```

Type:

```python
List[Document]
```

Each document:

```python
Document(
  page_content="text chunk",
  metadata={...}
)
```

Why Documents?

* Text for embeddings
* Metadata for filtering + citation

---

### ② `embedding`

```python
embeddings_1024
```

This is an **embedding model object**, NOT a vector.

Examples:

```python
OpenAIEmbeddings()
HuggingFaceEmbeddings()
GoogleGenerativeAIEmbeddings()
```

It must implement:

```python
embed_documents()
embed_query()
```

---

### ③ Optional keyword arguments (`**kwargs`)

These control **storage behavior**.

---

# 4️⃣ Full method signature (conceptual)

```python
Chroma.from_documents(
    documents: List[Document],
    embedding: Embeddings,
    persist_directory: str | None = None,
    collection_name: str = "langchain",
    client_settings: Settings | None = None,
    collection_metadata: dict | None = None
)
```

---

# 5️⃣ Different syntax cases (VERY IMPORTANT)

---

## ✅ Case 1: In-memory Chroma (temporary)

```python
db = Chroma.from_documents(
    final_documents,
    embeddings_1024
)
```

### Behavior:

* Stored in RAM
* Deleted when program ends
* Fast
* Good for:

  * testing
  * notebooks
  * experiments

---

## ✅ Case 2: Persistent Chroma (production)

```python
db = Chroma.from_documents(
    documents=final_documents,
    embedding=embeddings_1024,
    persist_directory="./chroma_db"
)
```

### Behavior:

* Stored on disk
* Reusable later
* Survives restarts

Later load it:

```python
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings_1024
)
```

---

## ✅ Case 3: Custom collection name

```python
db = Chroma.from_documents(
    final_documents,
    embeddings_1024,
    collection_name="speech_docs"
)
```

Why?

* Multiple datasets
* Clean separation

---

## ✅ Case 4: Add documents later

```python
db = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embeddings_1024
)

db.add_documents(new_docs)
```

Useful for:

* streaming data
* incremental ingestion

---

## ✅ Case 5: Metadata-based filtering

Your documents already contain:

```python
metadata = {"source": "speech.txt"}
```

You can query like:

```python
db.similarity_search(
    "democracy",
    filter={"source": "speech.txt"}
)
```

---

# 6️⃣ What is actually stored inside Chroma?

Each entry contains:

| Component        | Stored? |
| ---------------- | ------- |
| Embedding vector | ✅       |
| Text chunk       | ✅       |
| Metadata         | ✅       |
| Document ID      | ✅       |

Example (conceptual):

```json
{
  "id": "uuid",
  "embedding": [0.021, -0.93, ...],
  "text": "The world must be made safe...",
  "metadata": {
    "source": "speech.txt"
  }
}
```

---

# 7️⃣ Why embeddings dimension (1024) matters

Your variable:

```python
embeddings_1024
```

Means:

* Vector length = **1024**
* All vectors must have SAME dimension

If mismatch:
❌ Chroma throws error
❌ Retrieval breaks

---

# 8️⃣ How retrieval works later

```python
db.similarity_search("political liberty", k=3)
```

Steps:

1. Query → embedding
2. Compare with stored vectors
3. Use cosine similarity
4. Return top-k documents

---

# 9️⃣ Common mistakes (⚠️)

### ❌ Passing strings instead of Documents

```python
Chroma.from_documents(texts, embeddings)  # WRONG
```

### ❌ Passing vectors instead of embedding object

```python
Chroma.from_documents(docs, vectors)  # WRONG
```

### ❌ Forgetting persist_directory for production

```python
# Data lost on restart
```

---

# 🔟 Relationship to full RAG pipeline

```
Loader
  ↓
Documents
  ↓
Splitter
  ↓
Chunks (Documents)
  ↓
Embeddings
  ↓
Chroma VectorDB   ← YOU ARE HERE
  ↓
Retriever
  ↓
LLM
```

---

# 1️⃣1️⃣ Minimal end-to-end example (clean)

```python
db = Chroma.from_documents(
    documents=final_documents,
    embedding=embeddings_1024,
    persist_directory="./db",
    collection_name="speech"
)

retriever = db.as_retriever(search_kwargs={"k": 3})
```

---

# 🧠 One-line memory hook

> **`from_documents()` = embed + store + index in one call**

---

If you want next:

* `as_retriever()` parameters
* similarity vs MMR search
* how metadata filtering works internally
* cosine vs dot vs L2 in Chroma
* common tuning tricks for better retrieval accuracy

Just tell me 🚀


OLLAMA EMBEDDINGS

In [15]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.document_loaders import TextLoader
#install the model without installing the ollama model will not work
embeddings=OllamaEmbeddings(model='llama3.2:1b')


C:\Users\mydhi\AppData\Local\Temp\ipykernel_26572\1097847814.py:4: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaEmbeddings``.
  embeddings=OllamaEmbeddings(model='llama3.2:1b')


In [16]:
vectors_result=embeddings.embed_documents(['Alpha is the first greek character',
                            'Beta is the second beta character' ])
vectors_result


[[-2.2729716300964355,
  2.4434032440185547,
  -1.1681700944900513,
  -1.02479088306427,
  3.477818727493286,
  -0.3723914623260498,
  1.0390697717666626,
  3.1719796657562256,
  -2.888664960861206,
  4.691711902618408,
  -0.808150053024292,
  -2.524256467819214,
  -1.7176709175109863,
  0.8270727396011353,
  0.4385007619857788,
  -0.3426685333251953,
  -0.8586146235466003,
  4.500805854797363,
  2.6258459091186523,
  1.2409324645996094,
  -1.031480073928833,
  4.238842010498047,
  0.21900884807109833,
  3.3893561363220215,
  -4.540726661682129,
  -2.400266647338867,
  -2.624850034713745,
  1.6564738750457764,
  -1.4318933486938477,
  2.2302796840667725,
  0.9915040731430054,
  -1.934401273727417,
  -0.17955784499645233,
  0.06319563090801239,
  -2.6395912170410156,
  -2.413715362548828,
  1.0008357763290405,
  -0.8901791572570801,
  2.685455799102783,
  -1.3665804862976074,
  0.585006594657898,
  -2.144719123840332,
  -3.251253128051758,
  1.0775892734527588,
  -1.18894362449646,
  0.

In [17]:
len(vectors_result[0])

2048

In [19]:
res2=embeddings.embed_query(' second Greek Character is Beta')
res2

[-3.481583595275879,
 1.2190521955490112,
 -1.419120192527771,
 -0.23637382686138153,
 2.5208864212036133,
 -0.7928522825241089,
 1.533515453338623,
 0.8505315184593201,
 -0.32557520270347595,
 3.41660475730896,
 -1.4499098062515259,
 0.012454088777303696,
 -0.024077314883470535,
 -1.4277894496917725,
 -0.026595061644911766,
 0.4803408682346344,
 0.010546717792749405,
 1.9837591648101807,
 1.447118878364563,
 0.016940457746386528,
 1.1314926147460938,
 2.800093173980713,
 -1.45374596118927,
 2.2062442302703857,
 -2.0879673957824707,
 -0.19723039865493774,
 -0.6595844030380249,
 2.7384068965911865,
 -1.7305840253829956,
 -0.0976690948009491,
 -0.7601004242897034,
 -2.0668442249298096,
 -0.9244423508644104,
 0.10905923694372177,
 -1.0221906900405884,
 0.8140602111816406,
 0.020938517525792122,
 1.6645346879959106,
 1.6097304821014404,
 -4.3426513671875,
 0.49429136514663696,
 -0.08984887599945068,
 -3.84893536567688,
 1.1469594240188599,
 -4.9867424964904785,
 2.1920104026794434,
 0.6933

In [20]:
len(res2)

2048

In [22]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['HUGGINGFACE_API']=os.getenv('HUGGINGFACE_API')

In [33]:
from langchain_huggingface import HuggingFaceEmbeddings
hfembeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\mydhi\OneDrive\Desktop\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
text="this is a test document"
query_result=hfembeddings.embed_query(text)
query_result

[-0.048207856714725494,
 0.11789611726999283,
 -0.03746984153985977,
 0.056620463728904724,
 0.015501760877668858,
 -0.03674929589033127,
 -0.05957154184579849,
 0.05720916762948036,
 -0.020756378769874573,
 0.05708468705415726,
 0.07765141129493713,
 0.018936626613140106,
 0.0006362836575135589,
 -0.00040409056236967444,
 -0.06529419124126434,
 -0.02855006977915764,
 -0.011813539080321789,
 -0.04569057375192642,
 -0.00752564612776041,
 0.08929041028022766,
 0.053103718906641006,
 0.06305599212646484,
 -0.004552487749606371,
 0.0003609218983910978,
 0.008460181765258312,
 0.03009290061891079,
 -0.06308865547180176,
 0.03805892914533615,
 0.08158418536186218,
 -0.05816446617245674,
 0.03205757588148117,
 0.06851984560489655,
 0.07797116041183472,
 0.03416929394006729,
 0.06476923823356628,
 0.004228671081364155,
 0.07276926189661026,
 0.0025178862269967794,
 0.038405708968639374,
 0.035197965800762177,
 -0.01765652932226658,
 -0.11470245569944382,
 0.009409741498529911,
 0.0347809083759

In [35]:
len(query_result)

384

In [37]:
hfres2=hfembeddings.embed_documents([text,'this is not a test document'])
hfres2

[[-0.04820786416530609,
  0.11789610981941223,
  -0.03746983781456947,
  0.056620486080646515,
  0.015501742251217365,
  -0.03674928471446037,
  -0.05957150086760521,
  0.05720912665128708,
  -0.020756345242261887,
  0.05708470568060875,
  0.07765136659145355,
  0.018936704844236374,
  0.0006362390122376382,
  -0.0004040918138343841,
  -0.06529415398836136,
  -0.0285500455647707,
  -0.011813544668257236,
  -0.04569053649902344,
  -0.0075256479904055595,
  0.08929038792848587,
  0.0531037412583828,
  0.06305601447820663,
  -0.004552464000880718,
  0.0003609220730140805,
  0.008460141718387604,
  0.030092913657426834,
  -0.06308871507644653,
  0.03805888071656227,
  0.0815841406583786,
  -0.058164458721876144,
  0.03205760940909386,
  0.06851983815431595,
  0.07797116041183472,
  0.03416929766535759,
  0.0647692158818245,
  0.004228713922202587,
  0.07276924699544907,
  0.002517937682569027,
  0.03840569034218788,
  0.035197947174310684,
  -0.017656544223427773,
  -0.11470243334770203,
 